### Adjust Word Level

In [1]:
#!pip install xlsxwriter

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word

In [5]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)  # create path

##### Dependency DataFrames

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552149,fruitcocktail,6
552150,andthesunlightshining,6
552151,upravo,6
552152,yagawa,6


In [7]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,come on,1347415
1,thank you,1222974
2,all right,946314
3,i know,459647
4,excuse me,385487
...,...,...
726241,gwen turning,4
726242,gwen wait,4
726243,fucking xactos,4
726244,fucking yellow,4


In [8]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [9]:
#df_pair_ety = pd.read_excel(f"{lang_pair_list[0]}")  # need only dict_entry_main column
df_pair_ety = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
df_pair_ety

,dict_entry_main,turkish_word
0,abacus,abaküs
1,blockade,abluka
2,absorb,absorbe
3,absurd,absürt
4,azalea,açelya
...,...,...
1773,zebra,zebra
1774,zigzag,zikzak
1775,zombie,zombi
1776,zoology,zooloji


In [10]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair_merge = pd.merge(df_pair_ety,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair_merge.drop_duplicates(inplace=True)
    df_pair_merge.reset_index(drop=True, inplace=True)
    df_pair_merge = df_pair_merge.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair_merge.rename(columns={"word":"dict_entry_main"}, inplace=True)
    df_pair_all = df_pair_merge
else:
    df_pair_all = df_pair_ety

df_pair_all

,dict_entry_main,turkish_word
0,abacus,abaküs
1,blockade,abluka
2,absorb,absorbe
3,absurd,absürt
4,azalea,açelya
...,...,...
1773,zebra,zebra
1774,zigzag,zikzak
1775,zombie,zombi
1776,zoology,zooloji


In [11]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [12]:
words = df_pair_all["dict_entry_main"].values.tolist()
disable_set = set(disable_list)
words_set = set(words)

In [13]:
df_pair = pd.DataFrame(list(words_set.difference(disable_set)), columns=["dict_entry_main"])
df_pair

,dict_entry_main
0,anemia
1,manipulate
2,offside
3,lavender
4,station
...,...
1735,ace
1736,solution
1737,disinfectant
1738,route


##### Dependency Functions

In [14]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [15]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [16]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, word_thresh_num is an integer, column_list is list value
    '''
    word_count_bool(df, word_thresh_num, column_list):\n
    df is a dataframe, word_thresh_num is an integer, column_list is df column names\n
    word_count_bool(df, 7, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

#### Condition 1

In [17]:
def condition1(word_start_num = 0, word_limit_num = 28, thresh_num = 2, total_part_num = 4):
    '''
    default parameter:\n    
    condition1(word_start_num = 0, word_limit_num = 28, thresh_num = 2, total_part_num = 4) \n
    word_end 28/4 = 7 word group
    '''

    # while loop code block word and twogram pair
    word_thresh_num = thresh_num  # want how many word sample 
    word_start = word_start_num  # 0
    word_end = int((word_limit_num-word_start_num)/total_part_num)  
    step_num = word_end  
    word_limit = word_limit_num   
    part_num = 1  # first output file extention
    
    #twogram_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram
    twogram_pair_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram pair
    
    while word_end <= word_limit:
        df_word = df_word_all.iloc[word_start:word_end,]  # must be include word and frequency column
        df_word.reset_index(drop=True, inplace=True)
    
        # language pair twogram
        word_pair_list = df_pair["dict_entry_main"].to_list()  # *****
        word_list = df_word["word"].to_list()  # *****    
        ngram_list = []
        for i in word_pair_list:
            for j in word_tokenize(i):
                for k in word_list:
                    twogram_1_2 = f"{j} {k}"
                    ngram_list.append(twogram_1_2)
                    twogram_2_1 = f"{k} {j}"
                    ngram_list.append(twogram_2_1)
        df_pair_ngram = pd.DataFrame(ngram_list, columns=["twogram"])
        #df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)  # ******
        df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
        df_pair_ngram.drop_duplicates(inplace=True)
        df_pair_ngram.reset_index(drop=True, inplace=True)
        df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
        df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
        df_lang_pair_twogram.drop_duplicates(inplace=True)
        #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    
        # output
        df_output_result = pd.concat([df_word, df_lang_pair_twogram], axis=1)
    
        df_lesson_result = pd.DataFrame(columns=["word","freq_word",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
        a = 0
        #for i in range(0,110):
        for i in range(len(df_output_result)):  # *****
            # Insert words and their count 
            try:
                word = df_output_result.iloc[i,0]  # word 
                freq_word = df_output_result.iloc[i,1]  # word freq
                df_lesson_result.loc[i,"word"] = word
                df_lesson_result.loc[i,"freq_word"] = freq_word
            except:
                pass
            
            # Insert twogram pair
            try:
                var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    try:
                        while word_count_bool(df_lesson_result, word_thresh_num, [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                            a += 1
                            var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                            freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                            df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                            df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                        else:
                            pass
                    except:
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
                else:
                    pass
            except:
                pass
            a += 1
    
        df_lesson_word_count = word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])
        df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
        df_lesson_result = df_lesson_result.drop_duplicates()
        df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)
    
        word_start += step_num
        word_end += step_num
        part_num += 1 

#### Condition 2

In [18]:
# --- Parameter Part ---

# condition 1
condition1_word_start = 0
condition1_word_limit = 28
condition1_word_thresh_num = 2
condition1_total_part_num = 4  # 28/4 = 7 word group

# condition 2
condition1_dependency = True  # True, False
word_thresh_num = 2
twogram_thresh_minus = 0  # for optional twogram thresh number 
twogram_pair_thresh_minus = 0  # for optinal twogram pair thresh number.

word_start = 0  # 0
word_end = condition1_word_limit  # condition1_word_limit must be equal word_end (condition2 parameter)
step_num = word_end  # 10
word_limit = word_end  # 200
part_num = 1
    
if condition1_dependency:        
    # Read previous part result
    condition1(word_start_num = condition1_word_start, word_limit_num = condition1_word_limit, thresh_num = condition1_word_thresh_num, total_part_num = condition1_total_part_num)  # word_end 28/4 = 7 word group. Condition 1 parameters 
    df_part_all = pd.DataFrame()
    part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_*_Word_Step_*_Result*.xlsx")
    for i in part_result_file:
        df_var = pd.read_excel(f"{i}")
        df_part_all = pd.concat([df_part_all,df_var], axis=0)
    df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
    df_part_twogram_pair.reset_index(drop=True, inplace=True)
    set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])
else:
    set_part_twogram_pair = set([])  # option for skip Condition 1
# --- Parameter End ---

# while loop code block

twogram_num = word_thresh_num * step_num   # word_thresh_num*step_num minimum: for each word takes two twogram
twogram_pair_num = word_thresh_num * step_num  # word_thresh_num*step_num minimum: for each word takes two twogram pair

while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    word_pair_list = df_pair["dict_entry_main"].to_list()  # *****
    word_list = df_word["word"].to_list()  # ***** 
    ngram_list = []
    for i in word_pair_list:
        for j in word_tokenize(i):
            for k in word_list:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram = pd.DataFrame(ngram_list, columns=["twogram"])
    #df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)  # *****
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
    df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
    df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = pd.DataFrame(result_list2, columns=[f"{data_kind}"])  # *****
    #df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})  # *****
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_set_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0
    b = 0

    for i in range(len(df_output_result)):  # *****
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num - twogram_thresh_minus), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num - twogram_pair_thresh_minus), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [19]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair_turkish,freq_twogram_pair_turkish,word_count
0,you,102069964.0,i am,136931.0,like that,12872,4.0
1,i,94447074.0,i do,114801.0,hit me,9163,4.0
2,the,77481215.0,do not,102558.0,my brother,6974,4.0
3,to,58281119.0,and you,53582.0,my lady,6354,4.0
4,is,50852895.0,for what,53302.0,what time,5992,4.0
5,a,49726799.0,it is,49111.0,like me,5643,4.0
6,not,48510772.0,what for,47522.0,no time,3976,4.0
7,it,47050281.0,is it,31032.0,your brother,3897,4.0
8,that,35204210.0,not me,23708.0,no chance,3118,4.0
9,and,34806253.0,you will,13353.0,fix it,2727,4.0


In [20]:
#word_count_result(df_lesson_result, ["twogram"])

In [21]:
#word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])

#### Read Output File And Multi Write

In [22]:
try:
    writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')
    for i in range(1, (condition1_total_part_num+1)):        
        if condition1_dependency:
            df_part_var = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{int(condition1_word_limit/condition1_total_part_num)}_Result{i}.xlsx")
            df_part_var.to_excel(writer, sheet_name=f'Word_Part1{i}', index=False)            
        else:
            pass
except:
    pass

df_part_var2 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result1.xlsx")
df_part_var2.to_excel(writer, sheet_name=f'Word_Part21', index=False)        
writer.save()

#### Output File Word Order

In [23]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [24]:
try:
    writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')
    for i in range(1, (condition1_total_part_num+1)):        
        if condition1_dependency:
            df_part_var = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{int(condition1_word_limit/condition1_total_part_num)}_Result{i}.xlsx")
            df_part_var_order = word_in_twogram(df_part_var, "word", f"twogram_pair_{lang_pair.lower()}")
            df_part_var_order_join = df_part_var_order.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
            df_part_var_order_join.to_excel(writer2, sheet_name=f'Word_Part1{i}', index=False)          
        else:
            pass
except:
    pass

df_part_var2 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result1.xlsx")
df_word_order_21 = word_in_twogram(df_part_var2, "word", f"twogram")
df_word_order_212 = word_in_twogram(df_part_var2, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)        
writer2.save()

#### Copy Move And Delete

In [25]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_limit}_Word_*.xlsx")
output_file

['English_Turkish_28_Word_Step_7_Result1.xlsx',
 'English_Turkish_28_Word_Step_7_Result2.xlsx',
 'English_Turkish_28_Word_Step_7_Result3.xlsx',
 'English_Turkish_28_Word_Step_7_Result4.xlsx',
 'English_Turkish_28_Word_Step_28_Result1.xlsx',
 'English_Turkish_28_Word_Result_With_Frequency.xlsx',
 'English_Turkish_28_Word_Join_Result_Without_Frequency.xlsx']

In [26]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/3-Adjust Word Level/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [27]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass